In [6]:
# Download CBDB sqlite
!git clone https://github.com/cbdb-project/cbdb_sqlite.git

Cloning into 'cbdb_sqlite'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 31 (delta 3), reused 9 (delta 3), pack-reused 22
Unpacking objects: 100% (31/31), done.


In [7]:
# import glob
import os
latest_file = "./cbdb_sqlite/latest.7z"
latest_db = latest_file.split("/")[2].split(".")[0] + ".db"
print(latest_db)
if os.path.isfile(latest_db):
    os.remove(latest_db)

latest.db


In [8]:
# Explode latest database file
!pip install pyunpack
!pip install patool
from pyunpack import Archive
Archive(latest_file).extractall(".")

     |████████████████████████████████| 77 kB 4.0 MB/s 


In [9]:
# Create connection to database
import sqlite3
import pandas as pd
global CONN
database_file = os.path.basename(latest_file).split(".")[0]+".db"
CONN = sqlite3.connect(database_file)

In [10]:
addr_df_c_name_chn = pd.read_sql_query("SELECT c_name_chn FROM ADDR_CODES", CONN).drop_duplicates().dropna()
addr_df_c_alt_names = pd.read_sql_query("SELECT c_alt_names FROM ADDR_CODES", CONN).drop_duplicates().dropna()
addr_df = pd.concat([addr_df_c_name_chn, addr_df_c_alt_names], axis=1).stack().reset_index(drop=True).drop_duplicates().dropna().to_frame(name='addr_chn')
addr_df

,addr_chn
0,[信息缺乏]
1,[未詳]
2,
3,中華人民共和國
4,北京省市
...,...
13374,定戎
13375,義合
13376,積石軍
13377,敷文


In [12]:
# read in address type csv file
address_type_df = pd.read_csv('../cbdb_entity_address_types.csv')
address_type_list = address_type_df['name'].to_list()
print(len(address_type_list))

128


In [13]:
def recursively_remove_addr_type_from_text(res, text, address_type_list):
  if not text:
    return
  res.append(text)
  for word in address_type_list:
    if text.endswith(word):
      remains = text[:len(text)-len(word)]
      recursively_remove_addr_type_from_text(res, remains, address_type_list)
      break;

def remove_addr_type_from_text(text, address_type_list):
  res = []
  recursively_remove_addr_type_from_text(res, text, address_type_list)
  return res

In [14]:
# Test get_addr_tokens_from_text
input = "沿邊第一區備禦後千戶所左衛"
res = remove_addr_type_from_text(input, address_type_list)
print(res)

['沿邊第一區備禦後千戶所左衛', '沿邊第一區備禦後千戶所', '沿邊第一區']


In [15]:
def names_massage(df, column):
  # split line by ;
  df[column] = df[column].apply(lambda x: x.split(';'))
  df = df.explode(column).drop_duplicates().dropna()
  # remove row with 、
  df = df[df[column].str.contains('、')==False]
  # replace spaces with blank
  df[column] = df[column].str.replace("\\s", "", regex=True)
  # remove single-character row
  df = df[df[column].apply(lambda x: len(x)>1)]
  # remove the addresses which contain any English letters or any sort of brackets
  df = df[df[column].str.contains('.*[a-zA-Z]+.*')==False]
  df = df[df[column].str.contains('\(|\[|{|\)|\]|}|（|「|【|）|」|】')==False]
  # remove address type recursively
  df[column] = df[column].apply(lambda x: remove_addr_type_from_text(x, address_type_list))
  df = df.explode(column).drop_duplicates().dropna()
  # remove single-character 
  df = df[df[column].apply(lambda x: len(x)>1)]
  # remove row that's too long
  df = df[df[column].apply(lambda x: len(x)<14)]
  return df

In [16]:
filtered_addr_df = names_massage(addr_df, 'addr_chn')

In [17]:
filtered_addr_df

,addr_chn
3,中華人民共和國
4,北京省市
4,北京省
4,北京
5,天津省市
...,...
13375,義合
13376,積石軍
13377,敷文
13378,江淮等路


In [18]:
# Sort by descending order of the length
final_df = filtered_addr_df.sort_values(by="addr_chn", key=lambda x: -x.str.len())
final_df

,addr_chn
7501,沿河祐溪等處蠻夷軍民長官司
9786,碉門魚通等處管軍守鎮萬戶府
9770,松潘宕疊威茂州等處軍民安撫
7607,葛蠻雍真等處蠻夷軍民長官司
10316,固原靜寧隆德鎮原等處兵備道
...,...
6216,王朝
6217,高江
6223,保靜
6211,辰陽


In [19]:
final_df.to_csv('cbdb_entity_addresses.csv', index=False)

In [ ]:
# close the db
CONN.close()